In [ ]:
!pip install PyPDF2

## 1. Read in Resume

In [ ]:
import PyPDF2

In [ ]:
resume_file_path = 'YOUR RESUME FILE PATH'
#e.g. resume_file_path = './Nicholas Renotte - Resume.pdf' 
fhandle = open(resume_file_path, 'rb')

In [ ]:
pdfReader = PyPDF2.PdfFileReader(fhandle)

In [ ]:
pagehandle = pdfReader.getPage(0)

In [ ]:
text = pagehandle.extractText()

In [ ]:
# Strip out unwanted text
text = text.replace('o ','')
text = text.replace('|', '')

In [ ]:
text

## 2. Import and Authenticate to Service

In [ ]:
# Import Watson
from ibm_watson import PersonalityInsightsV3

# Import authenticator
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
# Creds for Personality Insights
apikey = 'YOUR API KEY HERE'
url = 'YOUR URL HERE'

In [ ]:
# Authenticate to PI service
authenticator = IAMAuthenticator(apikey)
personality_insights = PersonalityInsightsV3(
        version='2017-10-13', 
        authenticator=authenticator
)
personality_insights.set_service_url(url)

## 3. Extract Resume Personality Insights

In [ ]:
profile = personality_insights.profile(text, accept='application/json').get_result()
profile

In [ ]:
for personality in profile['personality']:
    print(personality['name'], personality['percentile'])

In [ ]:
for personality in profile['values']:
    print(personality['name'], personality['percentile'])

In [ ]:
for personality in profile['needs']:
    print(personality['name'], personality['percentile'])

## 4. Visualise

In [ ]:
# Import matplotlib 
from matplotlib import pyplot as plt
# Import seaborn
import seaborn as sns
# Import pandas
import pandas as pd

In [ ]:
# Visualise profiles 
needs = profile['needs']
result = {need['name']:need['percentile'] for need in needs}
df = pd.DataFrame.from_dict(result, orient='index')
df.reset_index(inplace=True)
df.columns = ['need', 'percentile']

In [ ]:
df.head()

In [ ]:
# Create Plot
plt.figure(figsize=(15,5))
sns.barplot(y='percentile', x='need', data=df).set_title('Needs')
plt.show()